**INSTALL LIBRARIES**

In [1]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMapWithTime
from folium.plugins import TimestampedGeoJson
from numpy import nan
import math
import datetime


import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import seaborn as sns

**READ DATA**

In [3]:
#pandas
inno_geocoded = pd.read_csv("inno_geocoded.csv")
#geopandas
gdf = gpd.GeoDataFrame(inno_geocoded, geometry=gpd.points_from_xy(inno_geocoded.lng, inno_geocoded.lat))

c:\Users\eric9\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (53) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


**MOVE longitude latitude to lng lat columns**

In [6]:
#still needs to be coded

**FILTER FOR INNO**

In [7]:
inno2 = gdf["inno_nr"] == "inno_02"
inno2 = gdf[inno2]
inno2 = inno2[inno2["lat"].notna()] # only needed, until step above is coded

#change datatype of createad at to datetime
inno2["created_at_td"] = pd.to_datetime(inno2["created_at"])

**ADD COLOR COLUMN TO DATAFRAME**

In [9]:
inno2["created_at_int"] = inno2["created_at_td"].apply(lambda x:x.toordinal())
norm = matplotlib.colors.Normalize(vmin= 735272,vmax= 737397, clip=True)
mapper = plt.cm.ScalarMappable(norm=norm, cmap=plt.cm.viridis) #viridis can be changed for other color palette
inno2["color"] = inno2["created_at_int"].apply(lambda x: mcolors.to_hex(mapper.to_rgba(x)))

**PREPARE LINES FOR TIMESTAMPED GEOJSON ANIMATION**

Coordinates

In [11]:
#nested list with coordinates defining start and end of each line
x_coords = inno2["geometry"].x.to_list()
y_coords = inno2["geometry"].y.to_list()

coordinate_list_1 = [[x, y] for x, y in zip(x_coords, y_coords)]
coordinate_list_2 = [[x, y] for x, y in zip(x_coords, y_coords)]

#remove first element(coordinate) of second list
del coordinate_list_2[0]

coordinate_list_1

coords = [list(i) for i in zip(coordinate_list_1, coordinate_list_2)]


Time

In [12]:
time = []
for i in inno2["created_at"]:
    sublist = [i,i]
    time.append(sublist)


Color

In [13]:
color = inno2["color"].to_list()

In [15]:
#list with dicts containing coordinates, date/time and color information for each line
lines = []
for i,j,k in zip(coords,time,color):
    
    new_item = {}
    new_item['coordinates'] = i
    new_item["dates"] = j
    new_item["color"] = k

    lines.append(new_item)

**VISUALISATION**

In [18]:
m = folium.Map([47.32, 8.50], zoom_start=2, tiles = "CartoDB dark_matter", control_scale = True)


features = [
    {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": line["coordinates"],
        },
        "properties": {
            "times": line["dates"],
            "style": {"color": line["color"]},
            "icon": "circle",
            "iconstyle": {"fillColor": line["color"], "fillOpacity": 1, "stroke": "true", "radius": 5}
        },
    }
    for line in lines
]

TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": features,
    },
    period="PT1M",
    add_last_point=True,
).add_to(m)

m